Ana Solbas Casajús

# STRUCTURE-BASED VIRTUAL SCREENING 

This notebook is based on the jupyter notebook provided by Catalina Arnaiz, Bioinformatic Research Asisstant at CBGP/UPM-INIA for the Structural Biology for Lead Discovery course of the MSc in Computational Biology.

_README_

_This guide is prepared to be run in either macOS or Linux, not in Windows. Executable files of the programs implemented are only available for these OS and thus, running this code in Windows will give rise to an error._

## 1. Protein Sanitization

In order to "sanitize" or clean the protein from undesireable ligands the LePhar molecular docking tool **LePro** will be implemented. LePro is designed to automatically add hydrogen atoms to proteins and/or nucleic acids by explicitely considering the protonation state of histidine. All crystal water, ions, small ligands and cofactors except HEM are also removed. 

LePro software download: http://www.lephar.com/software.htm 

In [1]:
! ./lepro_linux_x86 ./JAK1.pdb

In [2]:
! ./lepro_linux_x86 ./JAK2.pdb

In [3]:
! ./lepro_linux_x86 ./JAK3.pdb

Now we will have a "clean" protein pdb file automatically saved as *pro.pdb* and a configuration file that can be used with LeDock (just another method). 

**NOTE**: the running of this cell in Windows is difficult because no actual LePro executable is available for this OS. Nonetheless, you can always prep your protein in Chimera by adding H (Tools > Structure Editing > Add H) and running a structure optimizaiton protocol: 500 steepest descent steps + 200 conjugate gradient steps (Tools > Structure Editing > Minimize Structure). Then proceed to eliminate any excess molecules present in the structure (Select > "whatever" + Actions > Atoms > delete). Instead you can run this from you Virtual Machine.

## 2. Ligand Sanitization - Ligand battery prep

The protein now has a clean structure and H added to it, so it is time to think about the ligands we want to dock onto our protein. Once a list of putative ligands/interactors has been created they need to be prepared for the docking protocol. This is a simple step that can be done using two Python packages *rdkit* and *openbabel*. 

**NOTE**: I encourage you to use VSCode to edit Notebooks, Python or even Ruby scripts. VSCode makes the instalation of packages very simple (at least for Python). I have experienced that working through Anaconda is easier, as it offers the possibility to create environments. I like to create environments for different "jobs", I like to think of it as a way to organize my work in a better way - for protein docking protocols I have created a new *conda environment* containing the two packages mentioned above. 

The following code should be run on the terminal of a VSCode window that has been opened through Anaconda Navigator:

```
    conda create -c rdkit -n my-rdkit-env rdkit
    conda env list #verify the availability of the environments
    conda activate my-rdkit-env

    conda install -c conda-forge openbabel -n my-rdkit-env #instal openbabel in your newly created rdkit environment

```

In [1]:
#import packages
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from openbabel import pybel
from openbabel import openbabel

In [2]:
##ADD HYDROGENS TO THE COMPOUNDS
suppl = Chem.SDMolSupplier('./database/Pharmit_QueryResults.sdf') #read compound file
out = Chem.SDWriter('./database/Pharmit_QueryResults_Hyd.sdf') #create output file
for mol in suppl:
    molH = Chem.AddHs(mol, addCoords=True)
    out.write(molH)
out.close()

In [3]:
## CONVERT SDF FILE INTO MOL2 FILE
out=pybel.Outputfile(filename='./database/Pharmit_QueryResults_Hyd.mol2',format='mol2',overwrite=True) #open file to read and write
for mol in pybel.readfile("sdf","./database/Pharmit_QueryResults_Hyd.sdf"):
    mol.localopt(forcefield='mmff94s', steps=500) #Locally optimize the coordinates
    out.write(mol)
out.close()

## 3. Docking 

### 3.1. Docking coordinate calculation

Coordinates for the docking have been obtained using chimera. The size of the pocket has been set to 25 A to ensure that all the volume of the pocket is covered in the docking analysis.

### 3.2. Docking using Smina

Smina is run through a Linux or macOS executable file and just needs to be called adding the required and desired attributes. 

```
./smina.static -r /home/osboxes/docking_results/JAK1_lepro.pdb -l /home/osboxes/docking_results/database/Pharmit_QueryResults_Hyd.mol2 -o /home/osboxes/docking_results/output/JAK1_smina.sdf --center_x 12.037 --center_y 14.796 --center_z -12.553 --size_x 25 --size_y 25 --size_z 25 --exhaustiveness 8

```

As an output smina produces an sdf file containing the 3D coordinates of all docked molecules. You can then load this sdf file onto a pandas dataframe (as I have showed you before) and merge both lists by the library ID that is the comon aspect of both dataframes. Once you have done this, you should sort the docking results by value (most negative to most positive).

I have also run the docking analysis with the JAK2 and JAK3 proteins to compare the results and see if these molecules have more binding affinity for JAK1. 

```
./smina.static -r /home/osboxes/docking_results/JAK2_lepro.pdb -l /home/osboxes/docking_results/database/Pharmit_QueryResults_Hyd.mol2 -o /home/osboxes/docking_results/output/JAK2_smina.sdf --center_x 14.342 --center_y 10.219 --center_z 3.129 --size_x 25 --size_y 25 --size_z 25 --exhaustiveness 8

./smina.static -r /home/osboxes/docking_results/JAK3_lepro.pdb -l /home/osboxes/docking_results/database/Pharmit_QueryResults_Hyd.mol2 -o /home/osboxes/docking_results/output/JAK3_smina.sdf --center_x 0.957 --center_y 15.090 --center_z -3.652 --size_x 25 --size_y 25 --size_z 25 --exhaustiveness 8
```


#### JAK1

The binding affinity is expressed in kcal/mol.

In [2]:
#load docking results
smina = PandasTools.LoadSDF('./output/JAK1_smina.sdf', embedProps=True, molColName=None, removeHs=False)
smina

,minimizedAffinity,ID
0,-8.80502,CHEMBL4285366
1,-8.68869,CHEMBL4285366
2,-8.32162,CHEMBL4285366
3,-8.13958,CHEMBL4285366
4,-8.11429,CHEMBL4285366
...,...,...
337,-8.39612,CHEMBL78377
338,-8.38836,CHEMBL78377
339,-8.29203,CHEMBL78377
340,-7.71382,CHEMBL78377


In [3]:
smina = smina.astype({'minimizedAffinity': 'float'})
smina_sorted = smina.sort_values('minimizedAffinity')
smina_sorted.head()

,minimizedAffinity,ID
297,-10.46712,CHEMBL4468002
324,-10.08422,CHEMBL4460419
198,-9.95787,CHEMBL3717493
126,-9.89003,CHEMBL3715037
216,-9.88594,CHEMBL258838


In [4]:
JAK1_bestHits = smina_sorted.iloc[:5,]

#### JAK2

In [5]:
smina_JAK2 = PandasTools.LoadSDF('./output/JAK2_smina.sdf', embedProps=True, molColName=None, removeHs=False)
smina_JAK2

,minimizedAffinity,ID
0,-8.55599,CHEMBL4285366
1,-8.27783,CHEMBL4285366
2,-8.18572,CHEMBL4285366
3,-7.99415,CHEMBL4285366
4,-7.99204,CHEMBL4285366
...,...,...
337,-8.30908,CHEMBL78377
338,-8.29809,CHEMBL78377
339,-8.00556,CHEMBL78377
340,-7.97456,CHEMBL78377


In [6]:
smina_JAK2 = smina_JAK2.astype({'minimizedAffinity': 'float'})
smina_JAK2_sorted = smina_JAK2.sort_values('minimizedAffinity')
smina_JAK2_sorted.head()

,minimizedAffinity,ID
9,-9.90470,CHEMBL4293188
117,-9.77108,CHEMBL3716162
36,-9.69194,CHEMBL1630793
10,-9.65642,CHEMBL4293188
297,-9.65630,CHEMBL4468002


In [7]:
#affinity for the best JAK1 compound
smina_JAK2_sorted[smina_JAK2_sorted['ID']=='CHEMBL4468002'].head(1)

,minimizedAffinity,ID
297,-9.6563,CHEMBL4468002


It looks like, in general, the binding affinities are just slighlity worse than those for JAK1. In addition, the compound that shows the highest binding affinity for JAK1 has a lower affinity for JAK2.

#### JAK3

In [8]:
smina_JAK3 = PandasTools.LoadSDF('./output/JAK3_smina.sdf', embedProps=True, molColName=None, removeHs=False)
smina_JAK3

,minimizedAffinity,ID
0,-9.61479,CHEMBL4285366
1,-9.06337,CHEMBL4285366
2,-9.05552,CHEMBL4285366
3,-8.82060,CHEMBL4285366
4,-8.75996,CHEMBL4285366
...,...,...
337,-8.90150,CHEMBL78377
338,-8.89866,CHEMBL78377
339,-8.85206,CHEMBL78377
340,-8.79782,CHEMBL78377


In [9]:
smina_JAK3 = smina_JAK3.astype({'minimizedAffinity': 'float'})
smina_JAK3_sorted = smina_JAK3.sort_values('minimizedAffinity')
smina_JAK3_sorted.head()

,minimizedAffinity,ID
252,-10.78594,CHEMBL3669559
225,-10.66308,CHEMBL4448573
297,-10.66245,CHEMBL4468002
253,-10.60519,CHEMBL3669559
324,-10.58255,CHEMBL4460419


In [10]:
#affinity for the best JAK1 compound
smina_JAK3_sorted[smina_JAK3_sorted['ID']=='CHEMBL4468002'].head(1)

,minimizedAffinity,ID
297,-10.66245,CHEMBL4468002


Even though this was not our main objective, it looks like we have obtained higher affinities for JAK3 than for JAK1. In addition, the compound with a higer binding affinity for JAK1, have a slightly better affinity for JAK3

In [11]:
smina_JAK3_sorted[smina_JAK3_sorted['ID']=='CHEMBL3717493'].head(1) 

,minimizedAffinity,ID
198,-9.83063,CHEMBL3717493


The third best compound obtained for JAK1 in terms of binding affinity has a lower affinity for JAK3, but the different is too is small so we cannot conclude that this compound is selective for JAK1.

#### Binding affinity values for the three proteins

In [12]:
pd.merge(pd.merge(smina_sorted,smina_JAK2_sorted,on='ID'),smina_JAK3_sorted,on='ID')

,minimizedAffinity_x,ID,minimizedAffinity_y,minimizedAffinity
0,-10.46712,CHEMBL4468002,-9.6563,-10.66245
1,-10.46712,CHEMBL4468002,-9.6563,-10.52173
2,-10.46712,CHEMBL4468002,-9.6563,-10.52140
3,-10.46712,CHEMBL4468002,-9.6563,-10.10782
4,-10.46712,CHEMBL4468002,-9.6563,-10.04481
...,...,...,...,...
27697,-4.55569,CHEMBL3693610,-5.7260,-6.38313
27698,-4.55569,CHEMBL3693610,-5.7260,-6.35112
27699,-4.55569,CHEMBL3693610,-5.7260,-6.27984
27700,-4.55569,CHEMBL3693610,-5.7260,-6.21345


However, for each compound we just want the best binding affinities for the three proteins. 

In [13]:
affinity_df = pd.merge(pd.merge(smina_sorted.groupby(['ID'])['minimizedAffinity'].first().reset_index(),
                                smina_JAK2_sorted,on='ID').groupby(['ID','minimizedAffinity_x'])['minimizedAffinity_y'].first().reset_index(),
                    smina_JAK3_sorted,on='ID').groupby(['ID','minimizedAffinity_x','minimizedAffinity_y'])['minimizedAffinity'].first().reset_index()
affinity_df

,ID,minimizedAffinity_x,minimizedAffinity_y,minimizedAffinity
0,CHEMBL1630790,-9.64083,-9.38701,-9.65999
1,CHEMBL1630793,-9.26477,-9.69194,-8.88656
2,CHEMBL1630796,-8.40110,-8.53165,-9.30397
3,CHEMBL2152300,-9.11373,-8.70845,-8.76425
4,CHEMBL2314501,-9.82427,-9.64143,-9.65011
5,CHEMBL258838,-9.88594,-9.54526,-10.06003
6,CHEMBL3356065,-8.95291,-9.26651,-9.06928
7,CHEMBL3393341,-8.93047,-8.62446,-8.72113
8,CHEMBL3393342,-8.88368,-8.68330,-8.77302
9,CHEMBL3393349,-9.13224,-8.55207,-8.69959


In [14]:
affinity_df.columns = ['ID', 'JAK1_Affinity', 'JAK2_Affinity', 'JAK3_Affinity']
affinity_df = affinity_df.sort_values('JAK1_Affinity')

In [15]:
affinity_df.head()

,ID,JAK1_Affinity,JAK2_Affinity,JAK3_Affinity
33,CHEMBL4468002,-10.46712,-9.65630,-10.66245
32,CHEMBL4460419,-10.08422,-9.46408,-10.58255
21,CHEMBL3717493,-9.95787,-9.60064,-9.83063
14,CHEMBL3715037,-9.89003,-9.36894,-9.65881
5,CHEMBL258838,-9.88594,-9.54526,-10.06003


Finally, we are going to calculate the $K_d$ values as it can be defined as

$$K_d = e^{\frac{\Delta G}{RT}}$$

In [16]:
import numpy as np
Kd_df = affinity_df.loc[:, affinity_df.columns != 'ID'].transform(lambda x: np.exp(x/0.592))
Kd_df['ID'] = affinity_df['ID']
Kd_df.head()

,JAK1_Affinity,JAK2_Affinity,JAK3_Affinity,ID
33,2.095380e-08,8.242990e-08,1.506495e-08,CHEMBL4468002
32,4.000924e-08,1.140508e-07,1.724180e-08,CHEMBL4460419
21,4.952807e-08,9.055601e-08,6.140381e-08,CHEMBL3717493
14,5.554170e-08,1.339349e-07,8.208115e-08,CHEMBL3715037
5,5.592675e-08,9.943616e-08,4.167794e-08,CHEMBL258838


#### Find JAK1 selective proteins

The ligands with higher affinity towards JAK1 also have a high affinity for JAK3 and, therefore they are not selective for JAK1 as we wanted. Thus, now we are only going to select those ligands with a selectivity for JAK1 > 25%.

Selectivity can be calculated as $$\text{Selectivity} = 100*\frac{Kd_{i}-Kd_{JAK1}}{Kd_{JAK1}}$$ where $i$ would be JAK2 or JAK3.

In [17]:
#Calculate selectivity
Kd_df['Selectivity_JAK2'] = Kd_df.apply(lambda row: 
                                     100*(row.JAK2_Affinity-row.JAK1_Affinity)/row.JAK1_Affinity, 
                                     axis=1)
Kd_df['Selectivity_JAK3'] = Kd_df.apply(lambda row: 
                                     100*(row.JAK3_Affinity-row.JAK1_Affinity)/row.JAK1_Affinity, 
                                     axis=1)

In [18]:
#Get only the compounds with a selectivity higher than 25%
Kd_sel= Kd_df.loc[(Kd_df['Selectivity_JAK2']> 25) & 
                   (Kd_df['Selectivity_JAK3']> 25),].sort_values('JAK1_Affinity')
Kd_sel.columns = ['JAK1_Kd','JAK2_Kd','JAK3_Kd','ID','Selectivity_JAK2','Selectivity_JAK3']
Kd_sel

,JAK1_Kd,JAK2_Kd,JAK3_Kd,ID,Selectivity_JAK2,Selectivity_JAK3
14,5.554170e-08,1.339349e-07,8.208115e-08,CHEMBL3715037,141.142919,47.782930
4,6.206704e-08,8.452662e-08,8.329632e-08,CHEMBL2314501,36.185992,34.203778
9,1.997736e-07,5.322970e-07,4.148896e-07,CHEMBL3393349,166.450125,107.679887
3,2.061186e-07,4.087265e-07,3.719612e-07,CHEMBL2152300,98.296777,80.459797
35,2.373012e-07,3.311898e-07,3.219939e-07,CHEMBL4649816,39.565172,35.690002
7,2.809039e-07,4.710297e-07,4.000651e-07,CHEMBL3393341,67.683617,42.420657
15,4.461918e-07,1.068083e-06,1.000021e-06,CHEMBL3715062,139.377501,124.123494


In [19]:
Kd_sel.merge(affinity_df, on='ID')

,JAK1_Kd,JAK2_Kd,JAK3_Kd,ID,Selectivity_JAK2,Selectivity_JAK3,JAK1_Affinity,JAK2_Affinity,JAK3_Affinity
0,5.554170e-08,1.339349e-07,8.208115e-08,CHEMBL3715037,141.142919,47.782930,-9.89003,-9.36894,-9.65881
1,6.206704e-08,8.452662e-08,8.329632e-08,CHEMBL2314501,36.185992,34.203778,-9.82427,-9.64143,-9.65011
2,1.997736e-07,5.322970e-07,4.148896e-07,CHEMBL3393349,166.450125,107.679887,-9.13224,-8.55207,-8.69959
3,2.061186e-07,4.087265e-07,3.719612e-07,CHEMBL2152300,98.296777,80.459797,-9.11373,-8.70845,-8.76425
4,2.373012e-07,3.311898e-07,3.219939e-07,CHEMBL4649816,39.565172,35.690002,-9.03033,-8.83298,-8.84965
5,2.809039e-07,4.710297e-07,4.000651e-07,CHEMBL3393341,67.683617,42.420657,-8.93047,-8.62446,-8.72113
6,4.461918e-07,1.068083e-06,1.000021e-06,CHEMBL3715062,139.377501,124.123494,-8.65653,-8.13979,-8.17877
